In [1]:
# Import our dependencies
import pandas as pd
import numpy as np
from functools import reduce

In [2]:
#  Import and read the input csv
cgs_df = pd.read_csv('../../Clean_Data/Clean_CSV_Files/cia-government_systems.csv')

In [3]:
# Open the list of country name corrections
corrections_df = pd.read_csv('../../Clean_Data/master_country_list/country_name_corrections.csv', encoding='utf-8')

# Convert the corrections dataframe to a dictionary.
correction_dict = dict(zip(corrections_df['wrong'], corrections_df['correct']))

In [4]:
# Apply the correction dictionary to fix the known errors
cgs_df['country'] = cgs_df['country'].replace(correction_dict)

In [5]:
#  Import and read the input csv
sss_df = pd.read_csv('../../Clean_Data/Clean_CSV_Files/country_social_security_systems-list.csv', encoding='utf-8')

In [6]:
# Apply the correction dictionary to fix the known errors
sss_df['country'] = sss_df['country'].replace(correction_dict)

In [7]:
# Open the master list of countries
countries_df = pd.read_csv('../../Clean_Data/master_country_list/country_profile_urls.csv', encoding='utf-8')
countries_df = countries_df.drop(['profile_url'], axis=1)

In [8]:
master_countries = countries_df['country'].tolist()

In [9]:
# Define the range of years for the data
years = list(range(1990, 2023))

# Create a helper dataframe and merge with existing to ensure resulting df contains all countries and years
helper_df = pd.DataFrame({'country':np.repeat(master_countries, len(years)), 'Year':np.tile(years, len(master_countries))})

# Merge the helper DataFrame with the original data to ensure the data range is complete.
frames = [helper_df, cgs_df, sss_df]
complete_df = reduce(lambda left,right: pd.merge(left,right,how='left',on='country'),frames)

# Sort by 'Country' and 'Year'.
complete_df = complete_df.sort_values(['country','Year'], axis=0)

# Create a "country_year" column
complete_df['Country_Year'] = complete_df['country'] + '_' + complete_df['Year'].astype(str)

# Get a list of the column names:
cols = list(complete_df.columns)

# Move the 'Country_Year' columnt to head of list using index(), pop() and insert()
cols.insert(0, cols.pop(cols.index('Country_Year')))

# Use loc to reorder the DataFrame:
complete_df = complete_df.loc[:, cols]

In [10]:
complete_df

,Country_Year,country,Year,government_type,sss_depth,system_type
0,Albania_1990,Albania,1990,parliamentary republic,2.0,"Universal medical benefits,Social insurance (c..."
1,Albania_1991,Albania,1991,parliamentary republic,2.0,"Universal medical benefits,Social insurance (c..."
2,Albania_1992,Albania,1992,parliamentary republic,2.0,"Universal medical benefits,Social insurance (c..."
3,Albania_1993,Albania,1993,parliamentary republic,2.0,"Universal medical benefits,Social insurance (c..."
4,Albania_1994,Albania,1994,parliamentary republic,2.0,"Universal medical benefits,Social insurance (c..."
...,...,...,...,...,...,...
6100,Zimbabwe_2018,Zimbabwe,2018,presidential republic,1.0,Employer-liability system (cash sickness and m...
6101,Zimbabwe_2019,Zimbabwe,2019,presidential republic,1.0,Employer-liability system (cash sickness and m...
6102,Zimbabwe_2020,Zimbabwe,2020,presidential republic,1.0,Employer-liability system (cash sickness and m...
6103,Zimbabwe_2021,Zimbabwe,2021,presidential republic,1.0,Employer-liability system (cash sickness and m...


In [11]:
complete_df.to_csv('../../Clean_Data/Clean_CSV_Files/gov_systems-ss_systems-complete.csv', encoding='utf-8', index=False)